# March Madness Monte Carlo Simulation

Use information to try to predict the 2023 March Madness Men's and Women's Basketball Championships.

## Configure Notebook and Download Competition Items

In [1]:
from pathlib import Path

cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [2]:
from pathlib import Path
import os

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle:
    path = Path('../input/march-machine-learning-mania-2023')
    !pip install -Uqq fastai
else:
    import zipfile,kaggle
    path = Path('march-machine-learning-mania-2023')
    if not path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)

## Load and Preview Provided Data

In [3]:
import pandas as pd

# general datasets
cities = pd.read_csv(path/'Cities.csv')
conferences = pd.read_csv(path/'Conferences.csv')
samp_submission = pd.read_csv(path/'SampleSubmission2023.csv')

# men's datasets
m_conf_tourney_games = pd.read_csv(path/'MConferenceTourneyGames.csv')
m_game_cities = pd.read_csv(path/'MGameCities.csv')
m_massey_ordinals = pd.read_csv(path/'MMasseyOrdinals_thru_Season2023_Day128.csv')
m_tourney_compact_results = pd.read_csv(path/'MNCAATourneyCompactResults.csv')
m_tourney_detailed_results = pd.read_csv(path/'MNCAATourneyDetailedResults.csv')
m_tourney_seed_round_slots = pd.read_csv(path/'MNCAATourneySeedRoundSlots.csv')
m_tourney_seeds = pd.read_csv(path/'MNCAATourneySeeds.csv')
m_tourney_slots = pd.read_csv(path/'MNCAATourneySlots.csv')
m_reg_season_compact_results = pd.read_csv(path/'MRegularSeasonCompactResults.csv')
m_reg_season_detailed_results = pd.read_csv(path/'MSeasons.csv')
m_secondary_tourney_compact_results = pd.read_csv(path/'MSecondaryTourneyCompactResults.csv')
m_secondary_tourney_teams = pd.read_csv(path/'MSecondaryTourneyTeams.csv')
m_team_coaches = pd.read_csv(path/'MTeamCoaches.csv')
m_team_conferences = pd.read_csv(path/'MTeamConferences.csv')
m_teams = pd.read_csv(path/'MTeams.csv')
m_team_spellings = pd.read_csv(path/'MTeamSpellings.csv', encoding='latin-1')

# women's datasets
w_game_cities = pd.read_csv(path/'WGameCities.csv')
w_tourney_compact_results = pd.read_csv(path/'WNCAATourneyCompactResults.csv')
w_tourney_detailed_results = pd.read_csv(path/'WNCAATourneyDetailedResults.csv')
w_tourney_seeds = pd.read_csv(path/'WNCAATourneySeeds.csv')
w_tourney_slots = pd.read_csv(path/'WNCAATourneySlots.csv')
w_reg_season_compact_results = pd.read_csv(path/'WRegularSeasonCompactResults.csv')
w_reg_season_detailed_results = pd.read_csv(path/'WRegularSeasonDetailedResults.csv')
w_seasons = pd.read_csv(path/'WSeasons.csv')
w_team_conferences = pd.read_csv(path/'WTeamConferences.csv')
w_teams = pd.read_csv(path/'WTeams.csv')
w_team_spellings = pd.read_csv(path/'WTeamSpellings.csv', encoding='latin-1')

In [4]:
m_teams.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2023
1,1102,Air Force,1985,2023
2,1103,Akron,1985,2023
3,1104,Alabama,1985,2023
4,1105,Alabama A&M,2000,2023


# Process Overview

Use ML to submit predictions related to the [Kaggle 2023 March Madness Competition](https://www.kaggle.com/competitions/march-machine-learning-mania-2023).

Process steps:
1. Evaluate data and build a classifier to predict probability a team wins in a head-to-head matchup
2. Use model within a Monte Carlo simulation to simulate the tournament


### Sample Submission Format

In [5]:
# preview
samp_submission.head()

,ID,Pred
0,2023_1101_1102,0.5
1,2023_1101_1103,0.5
2,2023_1101_1104,0.5
3,2023_1101_1105,0.5
4,2023_1101_1106,0.5


Note: `Pred` is the probability that the first team won.
- `ID` is formatted such that the season, team_1, team_2 are presented

## Initial Data Preparation
### Tournament Datasets

#### Concatenate Men's and Women's Tournament Results

In [6]:
# add column to identify whether it is men's or women's 
m_tourney_compact_results['gender'] = 'men'
w_tourney_compact_results['gender'] = 'women'
m_tourney_detailed_results['gender'] = 'men'
w_tourney_detailed_results['gender'] = 'women'

# concat datasets together
tourney_compact = pd.concat([m_tourney_compact_results, w_tourney_compact_results], axis=0)
tourney_detailed = pd.concat([m_tourney_detailed_results, w_tourney_detailed_results], axis=0)

#### Add Conference Detail

In [7]:
# concat mens and womens datasets
team_conferences = pd.concat([m_team_conferences, w_team_conferences], axis=0)

In [8]:
# create copy of compact results dataset for now
df = tourney_compact.copy()

In [9]:
# add winning team conference detail
df = df.merge(team_conferences, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'])
df = df.rename(columns={'ConfAbbrev':'WTeamConf'})
df.drop(['TeamID'], axis=1, inplace=True)

# add losing team conference detail
df = df.merge(team_conferences, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'])
df = df.rename(columns={'ConfAbbrev':'LTeamConf'})
df.drop(['TeamID'], axis=1, inplace=True)

#### Concat City Detail

In [10]:
# concat mens and womens datasets
game_cities = pd.concat([m_game_cities, w_game_cities], axis=0)

In [11]:
# merge CRType and CityID into df
df = df.merge(game_cities, how='left', on=['Season', 'DayNum', 'WTeamID', 'LTeamID'])

#### Concat Tourney Seed Detail

In [12]:
# concat men's and women's datasets
tourney_seeds = pd.concat([m_tourney_seeds, w_tourney_seeds], axis=0)

In [13]:
# concat winning seed data
df = df.merge(tourney_seeds, how='left', left_on=['WTeamID', 'Season'], right_on=['TeamID', 'Season']).rename(columns={'Seed':'WTeamSeed'})
df = df.drop('TeamID', axis=1)

In [14]:
# concat losing seed data
df = df.merge(tourney_seeds, how='left', left_on=['LTeamID', 'Season'], right_on=['TeamID', 'Season']).rename(columns={'Seed':'LTeamSeed'})
df = df.drop('TeamID', axis=1)

#### Save Copy of Merged Tourney Data

In [15]:
tourney_df = df.copy()

### Regular Season Data

#### Concat Men's and Women's Regular Season Data

In [16]:
# add gender column to compact and detailed dataframes
m_reg_season_compact_results['gender'] = 'men'
m_reg_season_detailed_results['gender'] = 'men'
w_reg_season_compact_results['gender'] = 'women'
w_reg_season_detailed_results['gender'] = 'women'

In [17]:
# concat men's and women's regular season data
reg_season_compact = pd.concat([m_reg_season_compact_results, w_reg_season_compact_results], axis=0)
reg_season_detailed = pd.concat([m_reg_season_detailed_results, w_reg_season_detailed_results], axis=0)

#### Concat Conference Detail

In [18]:
team_conferences.head()

,Season,TeamID,ConfAbbrev
0,1985,1102,wac
1,1985,1103,ovc
2,1985,1104,sec
3,1985,1106,swac
4,1985,1108,swac


In [19]:
# save dataframe for compact reg season
df = reg_season_compact.copy()

In [20]:
# add winning team conference detail
df = df.merge(team_conferences, how='left', left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'])
df = df.rename(columns={'ConfAbbrev':'WTeamConf'})
df.drop(['TeamID'], axis=1, inplace=True)

# add losing team conference detail
df = df.merge(team_conferences, how='left', left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'])
df = df.rename(columns={'ConfAbbrev':'LTeamConf'})
df.drop(['TeamID'], axis=1, inplace=True)

#### Concat City Detail

In [21]:
# merge CRType and CityID into df
df = df.merge(game_cities, how='left', on=['Season', 'DayNum', 'WTeamID', 'LTeamID'])

#### Save Regular Season Data

In [22]:
reg_season_df = df.copy()

## Preprocess Datasets

In [23]:
# preview format of sample submissions
samp_submission.head()

,ID,Pred
0,2023_1101_1102,0.5
1,2023_1101_1103,0.5
2,2023_1101_1104,0.5
3,2023_1101_1105,0.5
4,2023_1101_1106,0.5


We know the Pred column is the probability that the first team beats the second team. 

In [24]:
# merge regular season and tournament data
df_concat = pd.concat([reg_season_df, tourney_df], axis=0)

In [25]:
df_concat.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,gender,WTeamConf,LTeamConf,CRType,CityID,WTeamSeed,LTeamSeed
0,1985,20,1228,81,1328,64,N,0,men,big_ten,big_eight,NaN,NaN,NaN,NaN
1,1985,25,1106,77,1354,70,H,0,men,swac,meac,NaN,NaN,NaN,NaN
2,1985,25,1112,63,1223,56,H,0,men,pac_ten,a_sun,NaN,NaN,NaN,NaN
3,1985,25,1165,70,1432,54,H,0,men,ivy,ind,NaN,NaN,NaN,NaN
4,1985,25,1192,86,1447,74,H,0,men,ecacm,ecacm,NaN,NaN,NaN,NaN


In [26]:
# create a copy of df_concat to work with
df_pre = df_concat.copy()

In [27]:
# add ID column matching sample submission
df_pre['ID_winner_first'] = df_pre.apply(lambda x: f'{x.Season}_{x.WTeamID}_{x.LTeamID}', axis=1)
df_pre['ID_loser_first'] = df_pre.apply(lambda x: f'{x.Season}_{x.LTeamID}_{x.WTeamID}', axis=1)

In [28]:
df_pre.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,gender,WTeamConf,LTeamConf,CRType,CityID,WTeamSeed,LTeamSeed,ID_winner_first,ID_loser_first
0,1985,20,1228,81,1328,64,N,0,men,big_ten,big_eight,NaN,NaN,NaN,NaN,1985_1228_1328,1985_1328_1228
1,1985,25,1106,77,1354,70,H,0,men,swac,meac,NaN,NaN,NaN,NaN,1985_1106_1354,1985_1354_1106
2,1985,25,1112,63,1223,56,H,0,men,pac_ten,a_sun,NaN,NaN,NaN,NaN,1985_1112_1223,1985_1223_1112
3,1985,25,1165,70,1432,54,H,0,men,ivy,ind,NaN,NaN,NaN,NaN,1985_1165_1432,1985_1432_1165
4,1985,25,1192,86,1447,74,H,0,men,ecacm,ecacm,NaN,NaN,NaN,NaN,1985_1192_1447,1985_1447_1192


In an effort to prevent overfitting to the format provided, we will divide the DataFrame in 2. One half will present winners first, with `Pred` set equal to 1. The remaining half will present losers first, with `Pred` set equal to 0. 

In [29]:
# split datasets in two
df_pre_a = df_pre.sample(frac=0.5)
df_pre_b = df_pre.drop(df_pre_a.index)

In [30]:
# reset indices
df_pre_a = df_pre_a.reset_index(drop=True)
df_pre_b = df_pre_b.reset_index(drop=True)

 - `df_pre_a` -- winner presented first within ID
 - `df_pre_b` -- loser presented first within ID

In [31]:
# create proper ID and Pred Columns
df_pre_a['ID'] = df_pre_a['ID_winner_first']
df_pre_a['Pred'] = 1
df_pre_b['ID'] = df_pre_b['ID_loser_first']
df_pre_b['Pred'] = 0

In [35]:
# drop helper columns
df_pre_a = df_pre_a.drop(['ID_winner_first', 'ID_loser_first'], axis=1)
df_pre_b = df_pre_b.drop(['ID_winner_first', 'ID_loser_first'], axis=1)

In [38]:
# merge datasets back together
df_preprocessed = pd.concat([df_pre_a, df_pre_b], axis=0)

In [39]:
df_preprocessed.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,gender,WTeamConf,LTeamConf,CRType,CityID,WTeamSeed,LTeamSeed,ID,Pred
0,1999,34,1243,78,1209,71,H,0,men,big_twelve,a_sun,NaN,NaN,NaN,NaN,1999_1243_1209,1
1,2016,57,1456,88,1141,84,H,0,men,caa,mac,Regular,4369.0,NaN,NaN,2016_1456_1141,1
2,1999,40,3375,76,3222,73,A,0,women,sun_belt,cusa,NaN,NaN,NaN,NaN,1999_3375_3222,1
3,2009,110,1171,66,1343,63,A,0,men,ivy,ivy,NaN,NaN,NaN,NaN,2009_1171_1343,1
4,2008,127,3288,86,3313,65,N,0,women,meac,meac,NaN,NaN,NaN,NaN,2008_3288_3313,1


Now that we have a combined dataset with the proper columns needed for submission, we can continue with EDA and begin building out our classification model. 

In [42]:
# save initial preprocessed dataset
data_files = path/'..'/'data'
df_preprocessed.to_csv(data_files/'initial_preprocessed.csv', index=False)